In [32]:
import pandas as pd
from ydata_profiling import ProfileReport
from IPython.display import IFrame

# Cargar dataset
df = pd.read_csv("UCI_Credit_Card.csv")

# Crear el reporte
profile = ProfileReport(
    df,
    title="Análisis Exploratorio - UCI Credit Card",
    explorative=True
)

# Guardar reporte en HTML
profile.to_file("Reporte_UCI_Credit_Card.html")

# Mostrar el HTML dentro del Notebook
IFrame(src="Reporte_UCI_Credit_Card.html", width=1000, height=600)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  5.23it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [37]:
# ------------------------------------------------------------
# 1. Importar librerías y cargar dataset con header reparado
# ------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

# --- Reparar el encabezado ---
with open("UCI_Credit_Card.csv", "r", encoding="utf-8", errors="ignore") as f:
    raw = f.readline()

raw = raw.replace('"""', '"').replace('""', '"').strip()
cols = next(csv.reader([raw], delimiter=",", quotechar='"'))

df = pd.read_csv("UCI_Credit_Card.csv", skiprows=1, names=cols)

# Arreglar nombre de ID si viene roto
if 'ID"' in df.columns:
    df = df.rename(columns={'ID"': 'ID'})

print("\nColumnas cargadas correctamente:")
print(df.columns.tolist())


# ------------------------------------------------------------
# 2. Eliminación condicional de duplicados por ID
# ------------------------------------------------------------
print("\n🔍 Verificando duplicados por ID...")

if 'ID' in df.columns:
    duplicados = df['ID'].duplicated().sum()
    print("Duplicados encontrados:", duplicados)

    if duplicados > 0:
        print("➡ Eliminando duplicados por ID...")
        df = df.drop_duplicates(subset=['ID'])
        print("Duplicados restantes:", df['ID'].duplicated().sum())
    else:
        print("✔ No hay duplicados por ID. No se elimina nada.")
else:
    print("⚠ La columna ID no existe en el dataframe.")


# ------------------------------------------------------------
# 3. Corrección condicional de valores inconsistentes
# ------------------------------------------------------------
print("\n🔍 Corrigiendo valores inconsistentes...")

# EDUCATION
if df['EDUCATION'].isin([0, 5, 6]).any():
    print("➡ Corrigiendo valores inválidos en EDUCATION (0, 5, 6 → 4)")
    df['EDUCATION'] = df['EDUCATION'].replace({0: 4, 5: 4, 6: 4})
else:
    print("✔ EDUCATION no tiene valores inconsistentes.")

# MARRIAGE
if df['MARRIAGE'].isin([0]).any():
    print("➡ Corrigiendo valores inválidos en MARRIAGE (0 → 3)")
    df['MARRIAGE'] = df['MARRIAGE'].replace({0: 3})
else:
    print("✔ MARRIAGE no tiene valores inconsistentes.")


# ------------------------------------------------------------
# 4. Ajuste condicional de tipos de datos
# ------------------------------------------------------------
print("\n🔍 Ajustando tipos de datos...")

cat_cols = [
    'SEX', 'EDUCATION', 'MARRIAGE',
    'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6',
    'default.payment.next.month'
]

for col in cat_cols:
    if col in df.columns:
        if df[col].dtype != "category":
            df[col] = df[col].astype('category')
            print(f"➡ Convertida a categoría: {col}")
    else:
        print(f"⚠ La columna {col} no existe.")


# ------------------------------------------------------------
# 5. Detección visual de outliers (no condicional)
# ------------------------------------------------------------
plt.figure(figsize=(6, 4))
df[['LIMIT_BAL']].boxplot()
plt.title("Boxplot - LIMIT_BAL")
plt.show()


# ------------------------------------------------------------
# 6. Winsorización condicional
# ------------------------------------------------------------
print("\n🔍 Winsorización en LIMIT_BAL (1% - 99%)...")

if df['LIMIT_BAL'].dtype in ['float64', 'int64']:
    q_low = df['LIMIT_BAL'].quantile(0.01)
    q_high = df['LIMIT_BAL'].quantile(0.99)
    df['LIMIT_BAL_wins'] = df['LIMIT_BAL'].clip(lower=q_low, upper=q_high)
    print("➡ LIMIT_BAL_wins creada correctamente.")
else:
    print("⚠ LIMIT_BAL no es numérica, no se aplica winsorización.")


# ------------------------------------------------------------
# 7. Creación condicional de etiquetas con map()
# ------------------------------------------------------------
# ================================================
# ETIQUETAS PARA VARIABLES CATEGÓRICAS
# ================================================

# 1. SEXO
df['SEX_LABEL'] = df['SEX'].map({
    1: 'Masculino',
    2: 'Femenino'
})

# 2. EDUCATION
df['EDUCATION_LABEL'] = df['EDUCATION'].map({
    1: 'Posgrado',
    2: 'Universidad',
    3: 'Secundaria',
    4: 'Otros/Desconocido'   # incluye 0, 5 y 6 que ya corregimos
})

# 3. MARRIAGE
df['MARRIAGE_LABEL'] = df['MARRIAGE'].map({
    1: 'Casado',
    2: 'Soltero',
    3: 'Otros'
})

# 4. Diccionario para todos los PAY_X
pay_map = {
    -2: 'No aplica / sin consumo',
    -1: 'Pago a tiempo',
    0:  'Pago dentro del mes',
    1:  'Retraso 1 mes',
    2:  'Retraso 2 meses',
    3:  'Retraso 3 meses',
    4:  'Retraso 4 meses',
    5:  'Retraso 5 meses',
    6:  'Retraso 6 meses',
    7:  'Retraso 7 meses',
    8:  'Retraso 8 meses',
    9:  'Retraso 9 meses o más'
}

pay_cols = ['PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']

for col in pay_cols:
    df[col + '_LABEL'] = df[col].map(pay_map)

# 5. Variable objetivo DEFAULT
df['DEFAULT_LABEL'] = df['default.payment.next.month'].map({
    0: 'No entra en default',
    1: 'Sí entra en default'
})

print("Etiquetas creadas correctamente.")



# ------------------------------------------------------------
# 8. Agregaciones condicionales
# ------------------------------------------------------------
# ================================================================
#     AGREGACIONES COMPLETAS PARA EL DATASET UCI CREDIT CARD
# ================================================================

print("\n================== 1. DEFAULT vs LIMIT_BAL ==================")
print(df.groupby('default.payment.next.month')['LIMIT_BAL'].mean())


print("\n================== 2. EDUCATION vs AGE =======================")
print(df.groupby('EDUCATION_LABEL')['AGE'].mean())


print("\n========== 3. Sexo vs Estado civil (conteo de clientes) ======")
print(df.groupby(['SEX_LABEL', 'MARRIAGE_LABEL'])['ID'].count())


print("\n================== 4. Suma de pagos por mes ==================")
print(df[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']].sum())


print("\n================== 5. Promedio de facturación mensual =========")
print(df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].mean())


print("\n====== 6. LIMIT_BAL por género, matrimonio y default =========")
print(df.groupby(['SEX_LABEL', 'MARRIAGE_LABEL', 'DEFAULT_LABEL'])['LIMIT_BAL'].mean())


print("\n=========== 7. Tabla dinámica EDUCATION vs DEFAULT ===========")
tabla = df.pivot_table(
    values='LIMIT_BAL',
    index='EDUCATION_LABEL',
    columns='DEFAULT_LABEL',
    aggfunc='mean'
)
print(tabla)


print("\n=============== 8. Distribución PAY_0 ========================")
print(df.groupby('PAY_0_LABEL')['ID'].count())



# ------------------------------------------------------------
# 9. Guardar dataset limpio
# ------------------------------------------------------------
df_clean = df.copy()
df_clean.to_csv("UCI_Credit_Card_clean.csv", index=False)
print("\n✔ Archivo limpio guardado como: UCI_Credit_Card_clean.csv")



Columnas cargadas correctamente:
['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default.payment.next.month']

🔍 Verificando duplicados por ID...
Duplicados encontrados: 0
✔ No hay duplicados por ID. No se elimina nada.

🔍 Corrigiendo valores inconsistentes...
➡ Corrigiendo valores inválidos en EDUCATION (0, 5, 6 → 4)
➡ Corrigiendo valores inválidos en MARRIAGE (0 → 3)

🔍 Ajustando tipos de datos...
➡ Convertida a categoría: SEX
➡ Convertida a categoría: EDUCATION
➡ Convertida a categoría: MARRIAGE
➡ Convertida a categoría: PAY_0
➡ Convertida a categoría: PAY_2
➡ Convertida a categoría: PAY_3
➡ Convertida a categoría: PAY_4
➡ Convertida a categoría: PAY_5
➡ Convertida a categoría: PAY_6
➡ Convertida a categoría: default.payment.next.month

🔍 Winsorización en LIMIT_BAL (1% 

C:\Users\cuno3\AppData\Local\Temp\ipykernel_15556\3622658627.py:91: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
C:\Users\cuno3\AppData\Local\Temp\ipykernel_15556\3622658627.py:175: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('default.payment.next.month')['LIMIT_BAL'].mean())
C:\Users\cuno3\AppData\Local\Temp\ipykernel_15556\3622658627.py:179: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('EDUCATION_LABEL')['AGE'].mean())
C:\Users\cuno3\AppData\Local\Temp\ipykernel_15556\3622658627.py:183: FutureWarning: The default of observed=Fals

SEX_LABEL  MARRIAGE_LABEL  DEFAULT_LABEL      
Masculino  Casado          No entra en default    205287.721124
                           Sí entra en default    150423.476969
           Soltero         No entra en default    154455.406472
                           Sí entra en default    104752.646465
           Otros           No entra en default    100194.174757
                           Sí entra en default     87380.952381
Femenino   Casado          No entra en default    187688.909063
                           Sí entra en default    137983.870968
           Soltero         No entra en default    174279.285242
                           Sí entra en default    130371.767241
           Otros           No entra en default    117729.729730
                           Sí entra en default     65744.680851
Name: LIMIT_BAL, dtype: float64

=========== 7. Tabla dinámica EDUCATION vs DEFAULT ===========
DEFAULT_LABEL      No entra en default  Sí entra en default
EDUCATION_LABEL              

# ✅ **III. Validación del Dataset Limpio**

Una vez realizadas las etapas de limpieza, corrección de valores, ajuste de tipos de datos, creación de etiquetas y generación de agregaciones, es fundamental evaluar si el dataset resultante conserva la calidad y estructura necesarias para responder a los objetivos del análisis. A continuación se validan los aspectos clave:

---

## **1. Completitud de los Datos**

Después del proceso de limpieza:

* No se detectaron valores nulos explícitos (`NaN`) en el dataset.
* Las variables que presentaban inconsistencias implícitas (por ejemplo, **EDUCATION = 0, 5, 6** o **MARRIAGE = 0**) fueron corregidas mediante recategorización, evitando la pérdida de datos.
* No se eliminaron filas por duplicados, ya que se comprobó que **no existían registros duplicados por `ID`**, lo cual garantiza que no hubo reducción innecesaria del tamaño del dataset.
* Después de aplicar **groupby()** para las agregaciones, se verificó que estas operaciones no alteran el dataset original, sino que producen resúmenes independientes. Por tanto, no hubo pérdida de observaciones.

✔ **Conclusión:** El dataset limpio es **completo**, no tiene ausencias relevantes y preserva la totalidad de registros válidos.

---

## **2. Relevancia de las Variables**

El dataset mantiene todas las variables críticas necesarias para analizar:

* El comportamiento crediticio del cliente
* El historial de pago por mes
* El límite de crédito aprobado
* El nivel educativo, edad, género y estado civil
* Montos facturados y montos pagados
* La variable objetivo **default.payment.next.month**

Además, se generaron nuevas **variables de etiqueta (LABEL)** que permiten análisis más interpretables:

* `SEX_LABEL`
* `EDUCATION_LABEL`
* `MARRIAGE_LABEL`
* `PAY_x_LABEL`
* `DEFAULT_LABEL`

Estas columnas no reemplazan a las originales sino que se agregan, por lo que **incrementan la relevancia analítica** sin modificar la estructura original.

✔ **Conclusión:** Las variables son pertinentes, completas y apropiadas para abordar las preguntas del análisis.

---

## **3. Granularidad Adecuada**

El dataset presenta una granularidad **a nivel de cliente**, donde cada fila representa:

> *“Un cliente único con su información demográfica, financiera y comportamiento de pago histórico.”*

Este nivel de granularidad es **exactamente el requerido** para:

* Modelar riesgo de default individual
* Analizar patrones de pago por cliente
* Segmentar clientes según comportamiento crediticio
* Estimar probabilidad de incumplimiento

No se realizaron agregaciones que modificaran la granularidad original del dataset.
Las operaciones de agregación (`groupby`, `pivot_table`) produjeron **resúmenes separados**, sin alterar la base principal.

✔ **Conclusión:** El dataset conserva una granularidad adecuada para análisis y modelos predictivos centrados en el cliente.

---

## **4. Evaluación Final de Adecuación del Dataset**

Basado en la revisión previa, se confirma que el dataset limpio:

* Preserva todos los registros válidos
* Mantiene todas las variables necesarias para los objetivos de negocio
* Agrega etiquetas interpretables sin afectar las variables originales
* Presenta valores corregidos y estandarizados
* Conservar la granularidad necesaria para análisis individual por cliente
* Elimina inconsistencias sin afectar la estructura analítica

✔ **El dataset final está completamente preparado para análisis exploratorio, modelado estadístico o machine learning.**

